This file generates the module

    * loadalot.py

In [1]:
%%writefile __init__.py
"""

"""

__version__ = '0.0.1' 

from . import xls
from . import tiff

### global variables ###

Overwriting __init__.py


## Import 

##tiff 

In [4]:
%%writefile tiff.py
#!/usr/bin/python
# -*- coding: latin-1 -*-

from PIL import Image
import numpy 
import string
import os, sys
import glob

def get_pixel_size(tifimage):
    """ 
    Returns: u'Image Pixel Size = 1.861 nm'
    
    Usage:
        from PIL import Image
        tifimage = Image.open(f)
    """
    item_4_1 = tifimage.tag.items()[4][1]
    lines = tifimage.tag.items()[4][1].split('\r\n')
    
    entry = None
    
    for line in lines:
        if 'Image Pixel Size' in line:
            line.encode('utf-8')
            entry = line
            break
        
    entry  = entry.split('=')[1].encode('utf-8')

    all_not_digits = string.letters + '.' + ' ' + 'µ'
    number = float(entry.translate(None, all_not_digits))
    dim    = entry.translate(None, string.digits + '.' + ' ')
    
    if dim == 'nm':
        number = number * 1e-9 
    elif dim == 'µm':
        number = number * 1e-6
    elif dim == 'mm':
        number = number * 1e-3
    
    return number

Overwriting tiff.py


In [5]:
%%writefile -a tiff.py

def scale_me(tiffile):
    """ 
    
    Usage:
        im = im.convert('RGB')
        imarray = numpy.array(im)

        x, y = scale_me(f)
        mpl.imshow(imarray, cmap = 'Reds', extent = [0, x, 0, y])
    """ 
    
    im = Image.open(tiffile)
    imarray = numpy.array(im)
    x, y = imarray.shape
    pixelsize = get_pixel_size(im)
    
    return y * pixelsize, x * pixelsize

Appending to tiff.py


## xls 

In [15]:
%%writefile xls.py

import openpyxl
from numpy import arange

def writecell(workbook, worksheet, row, column, content):
    """
    workbook  = xlsx filename
    worksheet = name
    row = number of the row as in the xlsx worksheet 
    column = str or index
    """
    #wb = workbook #openpyxl.load_workbook(workbook)
    #ws = wb.get_sheet_by_name(worksheet)
    wb = workbook
    ws = worksheet
    
    if type(column) == int:
        column = openpyxl.cell.get_column_letter(column)
    
    ws[column + str(row)] = content
    #wb.save(workbook)
    
def get_column_string_by_name(workbook, worksheet, column_name, row = 1):
    
    #wb = workbook #openpyxl.load_workbook(workbook)
    #ws = wb.get_sheet_by_name(worksheet)
    wb = workbook
    ws = worksheet
    
    max_col = ws.get_highest_column() 
    max_col = openpyxl.cell.get_column_letter(max_col)
    
    search_range = 'A'+str(row)+':'+max_col+str(row)
    
    for row in ws.iter_rows(search_range):
        for mycell in row:
            if mycell.value == column_name:
                xy = openpyxl.cell.coordinate_from_string(mycell.coordinate)
                return xy[0] 
            
def get_column_index_by_name(workbook, worksheet, column_name, row = 1):
    name = get_column_string_by_name(workbook, worksheet, column_name, row)
    return openpyxl.cell.column_index_from_string(name)
            
def get_row_index_by_name(workbook, worksheet, row_name, column = 'A'):
    
    #wb = workbook #openpyxl.load_workbook(workbook)
    #ws = wb.get_sheet_by_name(worksheet)
    wb = workbook
    ws = worksheet
    
    max_row = ws.get_highest_row() 
    #max_col = openpyxl.cell.get_column_letter(max_col)
    
    search_range = column+str(1)+':'+column+str(max_row)
    
    for row in ws.iter_rows(search_range):
        for mycell in row:
            if mycell.value == row_name:
                xy = openpyxl.cell.coordinate_from_string(mycell.coordinate)
                return xy[1] 

def load2dict(worksheet, dev_id):
    """
    :param worksheet: loaded worksheet of an xlrd workbook with one column 'ID' and other columns with more properties
        :code:
            workbook = xlrd.open_workbook('xls/A3172_S3_Measurements.xlsx')
            worksheet = workbook.sheet_by_index(0)
    :param dev_id: the name of the device for which will be searched in the column 'ID'
    :return: a dictionary with the names of the properties and the value of the device with the ID provided in 'dev_id'
    """
    column = worksheet.col_values(0) # ID

    ind0 = [i for i in arange(len(column)) if column[i] == 'ID']
    ind0 = int(ind0[0])
    keys = worksheet.row_values(ind0)

    ind = [i for i in arange(len(column)) if column[i] == dev_id]
    ind = int(ind[0])
    values = worksheet.row_values(ind)

    the_dict = {keys[i] : values[i] for i in arange(len(keys))}

    return the_dict

Overwriting xls.py


In [9]:
%%writefile -a xls.py

def table2xls(table, ):
    """ table = list of column-lists """

Appending to xls.py
